In [1]:
import os
import pandas as pd
from math import floor, ceil
from statistics import mean, median, mode

In [2]:
# Goal dataframe structure
# timestamp (min), speed, keystrokes, active window (app name), inactive time, PAM

# speed: average
# keystrokes: sum
# active window: mode
# inactive time: sum
# PAM: none (no duplicates in a minute)

# mousedata.tsv not needed

In [3]:
# PAM (photographic affect meter): a tool/app that measures emotion (aka affect)
# by showing photographs and asking users to select the one that best matches their mood

# Valence: pleasurable or not
# Arousal: activation or deactivation

#                 High Arousal
#             +------------------+
#             | 6 | 8 || 14 | 16 |
#             +------------------+
#             | 5 | 7 || 13 | 15 |
# Low Valence +==================+ High Valence
#             | 2 | 4 || 10 | 12 |
#             +------------------+
#             | 1 | 3 ||  9 | 11 |
#             +------------------+
#                 Low Arousal

# https://dl.acm.org/doi/10.1145/1978942.1979047



# looks like we can group values into buckets of 4
# (1, 2, 3, 4), (5, 6, 7, 8), (9, 10, 11, 12), (13, 14, 15, 16)

In [4]:
root_path = os.path.abspath('./archive/Data')

user1_path = os.path.join(root_path, 'user 1')
user2_path = os.path.join(root_path, 'user 2')

In [5]:
# read data from all files
def read_data(user_path: str, file_path: str):
    data = pd.read_csv(os.path.join(user_path, file_path), sep='\t')
    data.drop(data.filter(regex="Unnamed: "), axis=1, inplace=True)
    return data

activewindows_user1 = read_data(user1_path, 'activewindows.tsv')
inactivity_user1 = read_data(user1_path, 'inactivity.tsv')
keystrokes_user1 = read_data(user1_path, 'keystrokes.tsv')
mouse_mov_speeds_user1 = read_data(user1_path, 'mouse_mov_speeds.tsv')
#mousedata_user1 = read_data(user1_path, 'mousedata.tsv')
usercondition_user1 = read_data(user1_path, 'usercondition.tsv')

activewindows_user2 = read_data(user2_path, 'activewindows.tsv')
inactivity_user2 = read_data(user2_path, 'inactivity.tsv')
keystrokes_user2 = read_data(user2_path, 'keystrokes.tsv')
mouse_mov_speeds_user2 = read_data(user2_path, 'mouse_mov_speeds.tsv')
#mousedata_user2 = read_data(user2_path, 'mousedata.tsv')
usercondition_user2 = read_data(user2_path, 'usercondition.tsv')

In [6]:
# convert times to datetime type and floor to minutes
def convert_to_datetime(dataframe, index):
    return pd.to_datetime(dataframe.iloc[:,index]).dt.floor('30Min')

print(mouse_mov_speeds_user1.dtypes)

activewindows_user1.iloc[:,0] = convert_to_datetime(activewindows_user1, 0)

inactivity_user1.iloc[:,1] = convert_to_datetime(inactivity_user1, 1)
inactivity_user1.iloc[:,2] = convert_to_datetime(inactivity_user1, 2)

keystrokes_user1.iloc[:,1] = convert_to_datetime(keystrokes_user1, 1)
keystrokes_user1.iloc[:,2] = convert_to_datetime(keystrokes_user1, 2)

mouse_mov_speeds_user1.iloc[:,0] = convert_to_datetime(mouse_mov_speeds_user1, 0)

#mousedata_user1.iloc[:,0] = convert_to_datetime(mousedata_user1, 0)

usercondition_user1.iloc[:,0] = convert_to_datetime(usercondition_user1, 0)

# =========================================================

activewindows_user2.iloc[:,0] = convert_to_datetime(activewindows_user2, 0)

inactivity_user2.iloc[:,1] = convert_to_datetime(inactivity_user2, 1)
inactivity_user2.iloc[:,2] = convert_to_datetime(inactivity_user2, 2)

keystrokes_user2.iloc[:,1] = convert_to_datetime(keystrokes_user2, 1)
keystrokes_user2.iloc[:,2] = convert_to_datetime(keystrokes_user2, 2)

mouse_mov_speeds_user2.iloc[:,0] = convert_to_datetime(mouse_mov_speeds_user2, 0)

#mousedata_user2.iloc[:,0] = convert_to_datetime(mousedata_user2, 0)

usercondition_user2.iloc[:,0] = convert_to_datetime(usercondition_user2, 0)

print(mouse_mov_speeds_user1.dtypes)

Time          object
Speed(ms)    float64
Daylight      object
dtype: object
Time         datetime64[ns]
Speed(ms)           float64
Daylight             object
dtype: object


In [7]:
# we only care about mouse inactivity
print(inactivity_user1)

inactivity_user1 = inactivity_user1[inactivity_user1.Type == 'Mouse']

print(inactivity_user1)

          Type        Stopped_Time      Activated_Time  Duration(s)   Daylight
0        Mouse 2021-09-10 12:00:00 2021-09-10 12:00:00     6.255278  Afternoon
1        Mouse 2021-09-10 12:00:00 2021-09-10 12:00:00     5.773566  Afternoon
2        Mouse 2021-09-10 12:00:00 2021-09-10 12:00:00    10.883905  Afternoon
3        Mouse 2021-09-10 12:00:00 2021-09-10 12:00:00     5.108344  Afternoon
4        Mouse 2021-09-10 12:00:00 2021-09-10 12:00:00     6.071766  Afternoon
...        ...                 ...                 ...          ...        ...
2030     Mouse 2021-09-14 18:30:00 2021-09-14 18:30:00     6.928456    Evening
2031  Keyboard 2021-09-14 18:30:00 2021-09-14 18:30:00     5.315702    Evening
2032  Keyboard 2021-09-14 18:30:00 2021-09-14 18:30:00     9.716997    Evening
2033  Keyboard 2021-09-14 18:30:00 2021-09-14 18:30:00    18.270108    Evening
2034     Mouse 2021-09-14 18:30:00 2021-09-14 18:30:00     9.101637    Evening

[2035 rows x 5 columns]
       Type        Stopped_

In [8]:
# filter to desired columns
def filter_cols(dataframe, cols_to_keep):
    dataframe.drop(dataframe.columns.difference(cols_to_keep), axis=1, inplace=True)

print(mouse_mov_speeds_user1)

filter_cols(activewindows_user1, ['Time', 'App_Name'])
filter_cols(inactivity_user1, ['Activated_Time', 'Duration(s)'])
filter_cols(keystrokes_user1, ['Time', 'Press_Time', 'Relase_Time'])
filter_cols(mouse_mov_speeds_user1, ['Time', 'Speed(ms)'])
filter_cols(usercondition_user1, ['Time', 'PAM_Val'])

filter_cols(activewindows_user2, ['Time', 'App_Name'])
filter_cols(inactivity_user2, ['Activated_Time', 'Duration(s)'])
filter_cols(keystrokes_user2, ['Time', 'Press_Time', 'Relase_Time'])
filter_cols(mouse_mov_speeds_user2, ['Time', 'Speed(ms)'])
filter_cols(usercondition_user2, ['Time', 'PAM_Val'])

print(mouse_mov_speeds_user1)

                     Time  Speed(ms)   Daylight
0     2021-09-10 11:30:00   0.993728  Afternoon
1     2021-09-10 11:30:00   0.993729  Afternoon
2     2021-09-10 11:30:00   0.997684  Afternoon
3     2021-09-10 11:30:00   1.995367  Afternoon
4     2021-09-10 11:30:00   0.998678  Afternoon
...                   ...        ...        ...
19841 2021-09-14 18:30:00  14.920786    Evening
19842 2021-09-14 18:30:00   0.908916    Evening
19843 2021-09-14 18:30:00   1.183578    Evening
19844 2021-09-14 18:30:00   0.988831    Evening
19845 2021-09-14 18:30:00   0.995706    Evening

[19846 rows x 3 columns]
                     Time  Speed(ms)
0     2021-09-10 11:30:00   0.993728
1     2021-09-10 11:30:00   0.993729
2     2021-09-10 11:30:00   0.997684
3     2021-09-10 11:30:00   1.995367
4     2021-09-10 11:30:00   0.998678
...                   ...        ...
19841 2021-09-14 18:30:00  14.920786
19842 2021-09-14 18:30:00   0.908916
19843 2021-09-14 18:30:00   1.183578
19844 2021-09-14 18:30:00   

In [9]:
# shows that PAM has no duplicates in a minute
#duplicateRows = usercondition_user2[usercondition_user2.duplicated(['Time'])]
#print(duplicateRows)

# groups PAM values into groups of 4
# (1, 2, 3, 4), (5, 6, 7, 8), (9, 10, 11, 12), (13, 14, 15, 16) => (1), (2), (3), (4)

print(usercondition_user1.PAM_Val.value_counts())

usercondition_user1['PAM_Val'] = usercondition_user1['PAM_Val'].replace([1, 2, 3, 4], 1)
usercondition_user1['PAM_Val'] = usercondition_user1['PAM_Val'].replace([5, 6, 7, 8], 2)
usercondition_user1['PAM_Val'] = usercondition_user1['PAM_Val'].replace([9, 10, 11, 12], 3)
usercondition_user1['PAM_Val'] = usercondition_user1['PAM_Val'].replace([13, 14, 15, 16], 4)

usercondition_user2['PAM_Val'] = usercondition_user2['PAM_Val'].replace([1, 2, 3, 4], 1)
usercondition_user2['PAM_Val'] = usercondition_user2['PAM_Val'].replace([5, 6, 7, 8], 2)
usercondition_user2['PAM_Val'] = usercondition_user2['PAM_Val'].replace([9, 10, 11, 12], 3)
usercondition_user2['PAM_Val'] = usercondition_user2['PAM_Val'].replace([13, 14, 15, 16], 4)

print(usercondition_user1.PAM_Val.value_counts())

2     7
3     4
8     4
14    3
7     3
4     2
9     2
1     2
6     2
5     1
13    1
15    1
10    1
Name: PAM_Val, dtype: int64
1    15
2    10
4     5
3     3
Name: PAM_Val, dtype: int64


In [10]:
print('User 1')
print(usercondition_user1.PAM_Val.value_counts())
print()
print('User 2')
print(usercondition_user2.PAM_Val.value_counts())

User 1
1    15
2    10
4     5
3     3
Name: PAM_Val, dtype: int64

User 2
2    37
1    14
3    11
4     3
Name: PAM_Val, dtype: int64


In [11]:
# combines/aggregates values in the same minute
# side effect is that time is now the index, but can be fixed afterwards if needed

# speed: average
# keystrokes: sum
# active window: mode
# inactive time: sum
# PAM: none (no duplicates in a minute)

def median_rounded(input_list):
    median_list = median(input_list)
    mean_list = mean(input_list)

    if median_list < mean_list:
        median_list = ceil(median_list)
    else:
        median_list = floor(median_list)

    return median_list


print(inactivity_user1)

activewindows_user1 = activewindows_user1.groupby(activewindows_user1['Time']).aggregate({'App_Name': mode})
inactivity_user1 = inactivity_user1.groupby(inactivity_user1['Activated_Time']).aggregate({'Duration(s)': 'sum'})
inactivity_user1.columns = ['Inactivity Duration(s)']
keystrokes_user1 = keystrokes_user1.groupby(keystrokes_user1['Relase_Time']).aggregate({'Press_Time': 'count'})
keystrokes_user1.columns = ['Keystroke_Count']
mouse_mov_speeds_user1 = mouse_mov_speeds_user1.groupby(mouse_mov_speeds_user1['Time']).aggregate({'Speed(ms)': 'mean'})
usercondition_user1 = usercondition_user1.groupby(usercondition_user1['Time']).aggregate({'PAM_Val': median_rounded})

activewindows_user2 = activewindows_user2.groupby(activewindows_user2['Time']).aggregate({'App_Name': mode})
inactivity_user2 = inactivity_user2.groupby(inactivity_user2['Activated_Time']).aggregate({'Duration(s)': 'sum'})
inactivity_user2.columns = ['Inactivity Duration(s)']
keystrokes_user2 = keystrokes_user2.groupby(keystrokes_user2['Relase_Time']).aggregate({'Press_Time': 'count'})
keystrokes_user2.columns = ['Keystroke_Count']
mouse_mov_speeds_user2 = mouse_mov_speeds_user2.groupby(mouse_mov_speeds_user2['Time']).aggregate({'Speed(ms)': 'mean'})
usercondition_user2 = usercondition_user2.groupby(usercondition_user2['Time']).aggregate({'PAM_Val': median_rounded})

print(inactivity_user1)

          Activated_Time  Duration(s)
0    2021-09-10 12:00:00     6.255278
1    2021-09-10 12:00:00     5.773566
2    2021-09-10 12:00:00    10.883905
3    2021-09-10 12:00:00     5.108344
4    2021-09-10 12:00:00     6.071766
...                  ...          ...
2025 2021-09-14 18:30:00    12.297091
2026 2021-09-14 18:30:00    28.811897
2029 2021-09-14 18:30:00    13.008190
2030 2021-09-14 18:30:00     6.928456
2034 2021-09-14 18:30:00     9.101637

[1256 rows x 2 columns]
                     Inactivity Duration(s)
Activated_Time                             
2021-09-10 12:00:00              784.401936
2021-09-10 12:30:00              618.809407
2021-09-10 13:00:00             1048.829218
2021-09-10 13:30:00             1219.861964
2021-09-10 14:00:00              663.780873
2021-09-10 14:30:00              871.821915
2021-09-10 15:00:00             1585.753779
2021-09-10 15:30:00             1034.388266
2021-09-10 16:00:00              723.750394
2021-09-10 16:30:00             109

In [12]:
# combine (full outer join) all the dataframes together
comb_user1 = activewindows_user1.join(inactivity_user1).join(keystrokes_user1).join(mouse_mov_speeds_user1).join(usercondition_user1)
comb_user2 = activewindows_user2.join(inactivity_user2).join(keystrokes_user2).join(mouse_mov_speeds_user2).join(usercondition_user2)

print(comb_user1.columns)

Index(['App_Name', 'Inactivity Duration(s)', 'Keystroke_Count', 'Speed(ms)',
       'PAM_Val'],
      dtype='object')


In [13]:
# manually adding certain timestamps
# some sequences are missing 1 timestamp, which would typically split into 2 sequences in handle_missing_data()
# all_data = all_data.sort_index()
#comb_user2

In [14]:
# performs interpolation imputation based on contiguous* groups of time
# *allows a grace period with no timestamps
def handle_missing_data(df, timestamp_grace_period):
    '''
    timestamp_grace_period - int :
        how many missing timestamps are allowed before it counts as a new contiguous group of time.
        also adds these missing timestamps into the dataframe
    '''
    timedelta = pd.Timedelta(minutes=30)

    currDatetime = df.index[0]
    startDatetime = currDatetime
    endDatetime = currDatetime
    missing_timestamp_counter = 0

    while currDatetime != df.index[-1] + timedelta:
        if missing_timestamp_counter > 0:
            if currDatetime + timedelta in df.index:
                missing_timestamp_counter = 0
                currDatetime += timedelta
            else:
                missing_timestamp_counter += 1
                if missing_timestamp_counter > timestamp_grace_period:

                    before = df[:startDatetime - timedelta]
                    after = df[endDatetime:]
                    changed = df[startDatetime : endDatetime].resample('30Min').asfreq().interpolate(method='time', limit_direction='both').interpolate(method='pad')

                    df = pd.concat([before, changed, after])
                    
                    currDatetime += timedelta
                    startDatetime = currDatetime
                    missing_timestamp_counter = 0
                else:
                    currDatetime += timedelta
        else:
            if startDatetime in df.index:
                if currDatetime + timedelta in df.index:
                    currDatetime += timedelta
                elif currDatetime == df.index[-1]:
                    before = df[:startDatetime - timedelta]
                    changed = df[startDatetime:].resample('30Min').asfreq().interpolate(method='time', limit_direction='both').interpolate(method='pad')

                    df = pd.concat([before, changed])

                    currDatetime += timedelta
                else:
                    missing_timestamp_counter += 1
                    currDatetime += timedelta
                    endDatetime = currDatetime
            else:
                currDatetime += timedelta
                startDatetime = currDatetime

    return df


comb_user1 = handle_missing_data(comb_user1, 2)
comb_user1['Keystroke_Count'] = comb_user1['Keystroke_Count'].round()
comb_user1['PAM_Val'] = comb_user1['PAM_Val'].round()

comb_user2 = handle_missing_data(comb_user2, 2)
comb_user2['Keystroke_Count'] = comb_user2['Keystroke_Count'].round()
comb_user2['PAM_Val'] = comb_user2['PAM_Val'].round()
#comb_user2['App_Name'] = comb_user2['App_Name'].interpolate(method='pad')       # user 2 has some missing values in app name

In [15]:
print(comb_user1.head(10))

                           App_Name  Inactivity Duration(s)  Keystroke_Count  \
Time                                                                           
2021-09-10 11:30:00  TextLogger.exe              784.401936            296.0   
2021-09-10 12:00:00    explorer.exe              784.401936            296.0   
2021-09-10 12:30:00       opera.exe              618.809407           1048.0   
2021-09-10 13:00:00       opera.exe             1048.829218            280.0   
2021-09-10 13:30:00       opera.exe             1219.861964            348.0   
2021-09-10 14:00:00       opera.exe              663.780873            262.0   
2021-09-10 14:30:00       opera.exe              871.821915            458.0   
2021-09-10 15:00:00       opera.exe             1585.753779             57.0   
2021-09-10 15:30:00       opera.exe             1034.388266            171.0   
2021-09-10 16:00:00  TopTracker.exe              723.750394            259.0   

                     Speed(ms)  PAM_Val

In [16]:
print(comb_user2.head(10))

                         App_Name  Inactivity Duration(s)  Keystroke_Count  \
Time                                                                         
2021-09-07 08:30:00  explorer.exe             1333.010048              9.0   
2021-09-07 09:00:00  explorer.exe              424.366271             62.0   
2021-09-07 09:30:00  explorer.exe             5432.486898            114.0   
2021-09-07 10:00:00  explorer.exe             2498.585778             43.0   
2021-09-07 10:30:00  explorer.exe             2966.281775            145.0   
2021-09-07 11:00:00  explorer.exe             2369.555328            169.0   
2021-09-07 11:30:00  explorer.exe             2525.331417            144.0   
2021-09-07 12:00:00  explorer.exe             1676.141351            295.0   
2021-09-07 12:30:00  explorer.exe             2246.678371            136.0   
2021-09-07 13:00:00  explorer.exe             3291.911282            101.0   

                     Speed(ms)  PAM_Val  
Time                 

In [17]:
print('User 1:')
print(str(comb_user1.shape[0]) + ' timestamps')
print()
print('--- NaN Count ---')
print(comb_user1.isna().sum())
print()
print('--- PAM distribution ---')
print(comb_user1.PAM_Val.value_counts())

User 1:
40 timestamps

--- NaN Count ---
App_Name                  0
Inactivity Duration(s)    0
Keystroke_Count           0
Speed(ms)                 0
PAM_Val                   0
dtype: int64

--- PAM distribution ---
2.0    17
1.0    14
4.0     5
3.0     4
Name: PAM_Val, dtype: int64


In [18]:
print('User 2:')
print(str(comb_user2.shape[0]) + ' timestamps')
print()
print('--- NaN Count ---')
print(comb_user2.isna().sum())
print()
print('--- PAM distribution ---')
print(comb_user2.PAM_Val.value_counts())

User 2:
94 timestamps

--- NaN Count ---
App_Name                  0
Inactivity Duration(s)    0
Keystroke_Count           0
Speed(ms)                 0
PAM_Val                   0
dtype: int64

--- PAM distribution ---
2.0    59
1.0    18
3.0    13
4.0     4
Name: PAM_Val, dtype: int64


In [19]:
comb_user1.to_csv('./user1_preprocessed_2.csv')
comb_user2.to_csv('./user2_preprocessed_2.csv')